# BetExplorer odds

Prototype for collecting BetExplorer odds. This was tested both on matches with lots
of odds and matches with no odds at all.

In [1]:
from collections import namedtuple

import parsel
import requests


Odd = namedtuple('Odd', 'match_id, date, bookmaker, odd_type, odd_target, value')

## Tested matches

In [2]:
# typical match

match_id = 'hSbTPVSa'
url = 'http://www.betexplorer.com/gres/ajax/matchodds.php?p=1&e={}&b=1x2'.format(match_id)

headers = {
    'User-Agent': 'Secret Agent Bob',
    'Referer': 'http://www.betexplorer.com/soccer/brazil/serie-a-2017/atletico-go-fluminense/hSbTPVSa/',
}

In [14]:
# pumped up match

match_id = 'nTU1whIG'
url = 'http://www.betexplorer.com/gres/ajax/matchodds.php?p=1&e={}&b=1x2'.format(match_id)

headers = {
    'User-Agent': 'Secret Agent Bob',
    'Referer': 'http://www.betexplorer.com/soccer/europe/champions-league-2016-2017/juventus-real-madrid/nTU1whIG/',
}

In [4]:
# match without odds

match_id = 'GvDb0oPP'
url = 'http://www.betexplorer.com/gres/ajax/matchodds.php?p=1&e={}&b=1x2'.format(match_id)

headers = {
    'User-Agent': 'Secret Agent Bob',
    'Referer': 'http://www.betexplorer.com/soccer/tajikistan/vysshaya-liga-2012/zarafshon-panjakent-istiqlol-dushanbe/GvDb0oPP/',
}

## Core

In [15]:
r = requests.get(url, headers=headers)
r.status_code

200

In [16]:
body = r.json()['odds']
s = parsel.Selector(body)

In [17]:
odds = []

_rows = s.xpath('//tr[@data-originid]')
for _row in _rows:
    bookmaker = _row.css('a::text').extract_first()

    _odds = _row.css('td.table-main__odds')
    archive_odds = _odds.css('::attr(data-odd)').extract()
    archive_dates = _odds.css('::attr(data-created)').extract()
    opening_odds = _odds.css('::attr(data-opening-odd)').extract()
    opening_dates = _odds.css('::attr(data-opening-date)').extract()

    # archive odds
    odds.append(Odd(match_id, archive_dates[0], bookmaker, 'archive', '1', float(archive_odds[0])))
    odds.append(Odd(match_id, archive_dates[1], bookmaker, 'archive', 'X', float(archive_odds[1])))
    odds.append(Odd(match_id, archive_dates[2], bookmaker, 'archive', '2', float(archive_odds[2])))

    # opening odds
    odds.append(Odd(match_id, opening_dates[0], bookmaker, 'opening', '1', float(opening_odds[0])))
    odds.append(Odd(match_id, opening_dates[1], bookmaker, 'opening', 'X', float(opening_odds[1])))
    odds.append(Odd(match_id, opening_dates[2], bookmaker, 'opening', '2', float(opening_odds[2])))

In [18]:
len(odds)

162

In [20]:
[o._replace(match_id='') for o in odds[:6]]  # reduce line size

[Odd(match_id='', date='03,06,2017,20,25', bookmaker='10Bet', odd_type='archive', odd_target='1', value=2.95),
 Odd(match_id='', date='03,06,2017,20,07', bookmaker='10Bet', odd_type='archive', odd_target='X', value=3.1),
 Odd(match_id='', date='03,06,2017,20,27', bookmaker='10Bet', odd_type='archive', odd_target='2', value=2.7),
 Odd(match_id='', date='10,05,2017,22,47', bookmaker='10Bet', odd_type='opening', odd_target='1', value=2.85),
 Odd(match_id='', date='10,05,2017,22,47', bookmaker='10Bet', odd_type='opening', odd_target='X', value=3.15),
 Odd(match_id='', date='10,05,2017,22,47', bookmaker='10Bet', odd_type='opening', odd_target='2', value=2.5)]